## Deliverable 2. Create a Customer Travel Destinations Map.

In [8]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import os

# Import API key
from config import goo_API_key

# Configure gmaps API key
gmaps.configure(api_key=goo_API_key)

# checking API key, commented out so there is no output
# print(goo_API_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ljubinje,BA,42.9508,18.0886,27.35,74,0,3.26,clear sky
1,1,Mataura,NZ,-46.1927,168.8643,5.82,68,7,4.23,clear sky
2,2,Hermanus,ZA,-34.4187,19.2345,13.92,84,19,1.74,few clouds
3,3,Santiago De Cao,PE,-7.9589,-79.2392,16.07,82,40,5.66,fog
4,4,Busselton,AU,-33.6500,115.3333,13.94,86,92,10.24,light rain


In [ ]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [ ]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

In [ ]:
# 4a. Determine if there are any empty rows.

# replace empty strings with NaN
NaN_cities = preferred_cities_df.copy()
NaN_cities = NaN_cities.replace(r'^s*$',float('NaN'),regex=True)
NaN_cities.count()

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = NaN_cities.dropna()
clean_df.count()

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": goo_API_key,
    "location": ""
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = str(hotels["results"][0]["name"])
    except:
        print("Hotel not found... Skipping.")

#Check the dataframe        
hotel_df
        

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
# replace empty strings with NaN
NaN_hotel = hotel_df.copy()
NaN_hotel = NaN_hotel.replace(r'^s*$',float('NaN'),regex=True)
NaN_hotel

clean_hotel_df= NaN_hotel.dropna()

# check the dataframe again
clean_hotel_df

In [ ]:
# 8a. Create the output File (CSV)
clean_hotel_df = clean_hotel_df.reset_index(drop=True)
clean_hotel_df

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv")


In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [ ]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig